In [1]:
pwd

'/Users/aiden/Dropbox/Mac/Desktop/[minds]_transformers_bert/3.BERT를 이용한 질의 응답'

In [11]:
# !python -m pip install --upgrade pip

In [2]:
# !pip install tensorflow

In [10]:
# gpu 잡히는지 확인
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9308968656889551614
 xla_global_id: -1]

# 1. pipeline 이용해서

In [11]:
# !pip install transformers

In [12]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, QuestionAnsweringPipeline
import tensorflow as tf
import tqdm.notebook
import math

In [13]:
# !pip install torch

In [14]:
model = TFAutoModelForQuestionAnswering.from_pretrained('beomi/kcbert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# QA pipeline 세팅
qa = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

## 기본 kcbert 모델로 QA를 하면 어떨까

In [16]:
context = '윤여정은 1966년 연극 배우로 연기 경력을 시작하였고, 2021년 영화 《미나리》의 순자 역으로 아카데미 여우조연상을 수상했다.'

In [17]:
question = '윤여정의 직업은?'

In [18]:
qa(question=question, context=context)

{'score': 0.0032849288545548916,
 'start': 60,
 'end': 72,
 'answer': '여우조연상을 수상했다.'}

In [19]:
qa(question='윤여정은 2021년 무엇을 했나?', context=context)

{'score': 0.0032263416796922684,
 'start': 60,
 'end': 72,
 'answer': '여우조연상을 수상했다.'}

In [20]:
qa(question='윤여정의 미나리에서의 역할은?', context=context)

{'score': 0.0028909421525895596,
 'start': 60,
 'end': 72,
 'answer': '여우조연상을 수상했다.'}

# 2. korquad QA fine-tunning

In [21]:
model = TFAutoModelForQuestionAnswering.from_pretrained('beomi/kcbert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 데이터(KorQUAD) 전처리


In [22]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, QuestionAnsweringPipeline
import tensorflow as tf
import urllib.request
import json

In [23]:
# korquad 파일 다운로드
urllib.request.urlretrieve('https://korquad.github.io/dataset/KorQuAD_v1.0_train.json', 'korquad.json')

('korquad.json', <http.client.HTTPMessage at 0x7fee791f1ca0>)

In [24]:
korquad = json.load(open('korquad.json', encoding='utf8'))

In [25]:
# korquad 데이터셋 갯수
len(korquad['data'])

1420

In [26]:
korquad['data'][0]

{'paragraphs': [{'qas': [{'answers': [{'text': '교향곡', 'answer_start': 54}],
     'id': '6566495-0-0',
     'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'},
    {'answers': [{'text': '1악장', 'answer_start': 421}],
     'id': '6566495-0-1',
     'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?'},
    {'answers': [{'text': '베토벤의 교향곡 9번', 'answer_start': 194}],
     'id': '6566495-0-2',
     'question': '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?'},
    {'answers': [{'text': '파우스트', 'answer_start': 15}],
     'id': '6566518-0-0',
     'question': '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?'},
    {'answers': [{'text': '합창교향곡', 'answer_start': 354}],
     'id': '6566518-0-1',
     'question': '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?'},
    {'answers': [{'text': '1839', 'answer_start': 0}],
     'id': '5917067-0-0',
     'question': '바그너가 파우스트를 처음으로 읽은 년도는?'},
    {'answers': [{'text': '파리', 'answer_start': 410}],
     'id': '5917067-0-1',
     'question': '바그너가 처음 교향곡 작곡을 한 장소는?'},
    {'answers': [{'text': '드레스덴', 'answer_sta

In [27]:
para = korquad['data'][0]['paragraphs'][0]
para

{'qas': [{'answers': [{'text': '교향곡', 'answer_start': 54}],
   'id': '6566495-0-0',
   'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'},
  {'answers': [{'text': '1악장', 'answer_start': 421}],
   'id': '6566495-0-1',
   'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?'},
  {'answers': [{'text': '베토벤의 교향곡 9번', 'answer_start': 194}],
   'id': '6566495-0-2',
   'question': '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?'},
  {'answers': [{'text': '파우스트', 'answer_start': 15}],
   'id': '6566518-0-0',
   'question': '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?'},
  {'answers': [{'text': '합창교향곡', 'answer_start': 354}],
   'id': '6566518-0-1',
   'question': '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?'},
  {'answers': [{'text': '1839', 'answer_start': 0}],
   'id': '5917067-0-0',
   'question': '바그너가 파우스트를 처음으로 읽은 년도는?'},
  {'answers': [{'text': '파리', 'answer_start': 410}],
   'id': '5917067-0-1',
   'question': '바그너가 처음 교향곡 작곡을 한 장소는?'},
  {'answers': [{'text': '드레스덴', 'answer_start': 534}],
   'id': '5917067-0-2',
   'question': '바그너의 1

In [28]:
# 지문
para['context']

'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

In [29]:
# 질문과 답
qas = para['qas'][0]
qas

{'answers': [{'text': '교향곡', 'answer_start': 54}],
 'id': '6566495-0-0',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'}

In [30]:
inputs = tokenizer(para['context'], 
                   qas['question'], 
                   truncation=True, 
                   max_length=tokenizer.model_max_length,
                   return_tensors='tf')
inputs

{'input_ids': <tf.Tensor: shape=(1, 300), dtype=int32, numpy=
array([[    2,  8601,  4633, 29697,  1480,  4313,  4538,  4008,   336,
         4065,  4042,  3231, 23243,  4104,  4027,  8793, 13985,   391,
         9132,  4113, 10966, 11728, 12023, 14657,  4091,  8598, 16639,
          341,  4573,  4771,  4027,  2139,  8478, 14416,   214,  8202,
           17,  2451, 13007,  1480,  4313,  4538,  4008,  8601,  4633,
        22903,  4113,  1676,   868,  4913,  7965,  1789,  4203,  4110,
        15031,   786,   250,  4057, 10878,  4007,  2593,  4094,  4128,
        10289,  4113, 10958,  4062,  9511,  1355,  4600,  4103,  4775,
         5602, 10770,  4180, 26732,  3231, 23243,  4104,  4042,  2015,
         4012,  4113,  9198,  8763,  8129,    17, 10384, 23008,  7971,
         2170,  4408,  4011,  4147,  4042, 17015,  4091, 23008, 21056,
         4165,   323,  4175,  4158, 11413,  2273,  4043,  7966,  1543,
         4775,  4170,  4042,   341,  4573,  4771,    28,  4566,  4027,
        10599, 

In [31]:
q = qas['answers'][0]

# 글자의 위치
start_char = q['answer_start']
end_char = start_char + len(q['text']) - 1

# 토큰의 위치
start = inputs.char_to_token(0, start_char)
end = inputs.char_to_token(0, end_char)

In [32]:
# 글자의 위치
print(start_char)
print(end_char)

54
56


In [33]:
# 토큰의 위치
print(start)
print(end)

27
29


In [34]:
inputs['input_ids']

<tf.Tensor: shape=(1, 300), dtype=int32, numpy=
array([[    2,  8601,  4633, 29697,  1480,  4313,  4538,  4008,   336,
         4065,  4042,  3231, 23243,  4104,  4027,  8793, 13985,   391,
         9132,  4113, 10966, 11728, 12023, 14657,  4091,  8598, 16639,
          341,  4573,  4771,  4027,  2139,  8478, 14416,   214,  8202,
           17,  2451, 13007,  1480,  4313,  4538,  4008,  8601,  4633,
        22903,  4113,  1676,   868,  4913,  7965,  1789,  4203,  4110,
        15031,   786,   250,  4057, 10878,  4007,  2593,  4094,  4128,
        10289,  4113, 10958,  4062,  9511,  1355,  4600,  4103,  4775,
         5602, 10770,  4180, 26732,  3231, 23243,  4104,  4042,  2015,
         4012,  4113,  9198,  8763,  8129,    17, 10384, 23008,  7971,
         2170,  4408,  4011,  4147,  4042, 17015,  4091, 23008, 21056,
         4165,   323,  4175,  4158, 11413,  2273,  4043,  7966,  1543,
         4775,  4170,  4042,   341,  4573,  4771,    28,  4566,  4027,
        10599, 18907,   208, 

In [35]:
# 정답을 실제 텍스트로 변환
tokenizer.decode(inputs['input_ids'].numpy()[0, start:end+1])

'교향곡'

In [36]:
inputs['start_positions'] = [start]
inputs['end_positions'] = [end]

## TFRecord로 저장


In [37]:
def int_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def make_inputs(context, qas):
    inputs = tokenizer(
        context, 
        qas['question'], 
        truncation=True, 
        max_length=tokenizer.model_max_length)
    q = qas['answers'][0]
    start_char = q['answer_start']
    end_char = start_char + len(q['text']) - 1
    start = inputs.char_to_token(0, start_char)
    end = inputs.char_to_token(0, end_char)
    inputs['start_positions'] = [start]
    inputs['end_positions'] = [end]
    return inputs

In [38]:
%%time
n = 0
filename = 'korquad.tfrecord'
with tf.io.TFRecordWriter(filename) as writer:
    for item in korquad['data']:   # 각 아이템 순환
        for para in item['paragraphs']:  # 아이템마다 
            context = para['context']
            for qas in para['qas']:
                inputs = make_inputs(context, qas)
                if inputs['start_positions'][0] and inputs['end_positions'][0]:
                    feature = {k: int_feature(v) for k, v in inputs.items()}
                    example = tf.train.Example(features=tf.train.Features(feature=feature))
                    s = example.SerializeToString()
                    writer.write(s)
                    n += 1


CPU times: user 57.4 s, sys: 638 ms, total: 58 s
Wall time: 59 s


In [39]:
# 사례 수
n

57653

## 불러오기

In [40]:
int_seq = tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True)
int_value = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
feature_description = {
    'input_ids': int_seq,
    'token_type_ids': int_seq,
    'attention_mask': int_seq,
    'start_positions': int_value,
    'end_positions': int_value
}


In [41]:
def preproc(example):
    example = tf.io.parse_single_example(example, feature_description)
    example = {k : tf.cast(v, tf.int32) for k, v in example.items()}
    return example

In [42]:
# korquad 데이터셋 불러오기
dataset = tf.data.TFRecordDataset(['korquad.tfrecord']).map(preproc).padded_batch(8)
batch = next(iter(dataset))
result = model(batch)

## 수동으로 학습

In [ ]:
%%time
# 옵티마이저는 adam
opt = tf.keras.optimizers.Adam(learning_rate=5e-5)

n = 57653
for batch in tqdm.notebook.tqdm(dataset, total=math.ceil(n / 32)):
    with tf.GradientTape() as tape:
        result = model(batch)
        loss = tf.reduce_mean(result['loss'])
    grads = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables)) # loss가 감소하는 방향으로 파라미터 업데이트

  0%|          | 0/1802 [00:00<?, ?it/s]

In [86]:
model

In [62]:
context = '윤여정은 1966년 연극 배우로 연기 경력을 시작하였고, 2021년 영화 《미나리》의 순자 역으로 아카데미 여우조연상을 수상했다.'

In [43]:
def qa2(question, context):
    inputs = tokenizer(context, question, add_special_tokens=True, return_tensors="tf")
    outputs = model(inputs)
    start = tf.argmax(outputs.start_logits, axis=1).numpy()[0]
    end = tf.argmax(outputs.end_logits, axis=1).numpy()[0]
    return tokenizer.decode(inputs['input_ids'][0, start:end+1])

In [63]:
qa2('윤여정은 언제 연기를 시작했나?', context)

'1966년'

In [64]:
qa2('윤여정은 언제 상을 받았나?', context)

'2021년'

In [65]:
qa2('윤여정의 직업은?', context)

'연극 배우'

In [67]:
qa2('윤여정이 2021년 받은 상은?', context)

'순자 역으로 아카데미 여우조연상을'

In [68]:
qa2('윤여정의 2021년 출연작은?', context)

'미나리'

In [71]:
qa2('윤여정이 미나리에서 맡았던 역할은?', context)

'순자'

In [70]:
qa2('윤여정이 미나리에서 맡았던 역은?', context)

'순자'

In [72]:
qa2('윤여정은 2021년 무엇을 했는가?', context)

'미나리'

In [73]:
qa2('1966년 연극 배우로 연기 경력을 시작한 사람은 누구인가?', context)

'윤여정은'

In [76]:
qa2('미나리로 상을 받은 사람은 누구인가?', context)

'윤여정은'